In [19]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [20]:
# Load dataset
dataset = pd.read_csv('Traffic_updated.csv')

In [21]:
# Extract numerical features for training the GAN
numerical_features = dataset.select_dtypes(include=[np.number])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numerical_features)
scaled_data_shape = scaled_data.shape[1]

In [22]:
# Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(scaled_data_shape, activation='tanh'))
    return model

In [23]:
# Discriminator model
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [24]:
# GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [25]:
# Compile the models
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator(scaled_data_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [26]:
# Training the GAN
epochs = 1000
batch_size = 32

In [27]:
for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generated_data = generator.predict(noise)
    
    idx = np.random.randint(0, scaled_data.shape[0], batch_size)
    real_data = scaled_data[idx]
    
    labels_real = np.ones((batch_size, 1))
    labels_fake = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_data, labels_real)
    d_loss_fake = discriminator.train_on_batch(generated_data, labels_fake)

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    labels_gan = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, labels_gan)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

1/1 [==============================] - 0s 76ms/step
Epoch 0, D Loss: 0.7058361768722534, G Loss: 0.6892129778862
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 4ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step


In [28]:
# Generate synthetic data
num_samples = len(dataset)
noise = np.random.normal(0, 1, (num_samples, latent_dim))
generated_samples = generator.predict(noise)

93/93 [==============================] - 0s 2ms/step


In [29]:
# Inverse transform the scaled data to the original scale
generated_samples = scaler.inverse_transform(generated_samples)

In [30]:
# Create a DataFrame with the generated data
generated_df = pd.DataFrame(generated_samples, columns=numerical_features.columns)

In [31]:
# Save the generated data to a CSV file
generated_df.to_csv('generated_traffic_data.csv', index=False)

In [36]:
generated_data = pd.read_csv('generated_traffic_data.csv')

In [38]:
generated_data

,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation,hour,minute,AM/PM
0,17.093283,4.309733,55.342125,4.162469,12.290577,23.713167,172.84410,2.074958,20.022160,24.080511,0.871381
1,-1.402301,4.151830,76.498040,25.483230,27.001867,10.161179,115.50059,2.514740,8.649772,19.273218,0.149803
2,16.337072,4.587021,-37.996754,10.285139,-2.235146,-21.683790,137.55974,0.263116,7.936231,9.073615,0.614285
3,27.111027,2.554917,56.995660,28.169474,33.621960,5.509882,185.42818,0.953997,20.384853,26.311806,0.943056
4,12.423042,2.487404,47.365500,6.213130,13.579367,2.342750,145.37291,0.024522,7.586169,11.707626,0.683505
...,...,...,...,...,...,...,...,...,...,...,...
2971,8.836069,4.882486,154.260510,16.286660,26.561686,3.603023,228.08286,2.377446,5.579543,-0.313350,-0.126384
2972,12.402778,4.607894,148.944560,39.433445,30.356160,7.146340,186.14750,1.344280,15.248916,-3.372027,0.785758
2973,14.267838,3.710961,67.412440,4.291900,-13.711877,22.402225,-36.65427,0.796957,2.941027,17.598087,0.116725
2974,6.121564,5.160987,115.960510,18.264032,1.754553,13.635783,107.45341,0.981586,-7.043658,-12.453113,0.203029
